In [1]:
import torch
import torch.nn as nn
import numpy as np
from utilis_ForsaV2 import *
import utilis_ForsaV2
# !pip install fxpmath\
from fxpmath import Fxp
import time
word_size  = 8
frac_size = 6
utilis_ForsaV2.word_size = word_size
utilis_ForsaV2.frac_size = frac_size
# from tensorboardX import SummaryWriter
# from transforms import *
# from models_cust import *


In [2]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
model.eval()

Using cache found in C:\Users\huruy/.cache\torch\hub\pytorch_vision_v0.10.0


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [3]:
# # Uncomment to visualize it in netron locally
# # Assuming `model` is your PyTorch model
# model_path = './PretrainedModels/alexnetPretrainedIM1K.pth'

# # # Save the entire model, including its architecture and trained parameters
# # # torch.save(model, model_path)
# torch.jit.script(model).save(model_path)

In [4]:
# def count_parameters(model):
#     return sum(p.numel() for p in model.parameters() if p.requires_grad)
# print(count_parameters(model))

In [5]:
from torchsummary import summary
summary(model.to('cuda'), (3, 224, 224)) 

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 55, 55]          23,296
              ReLU-2           [-1, 64, 55, 55]               0
         MaxPool2d-3           [-1, 64, 27, 27]               0
            Conv2d-4          [-1, 192, 27, 27]         307,392
              ReLU-5          [-1, 192, 27, 27]               0
         MaxPool2d-6          [-1, 192, 13, 13]               0
            Conv2d-7          [-1, 384, 13, 13]         663,936
              ReLU-8          [-1, 384, 13, 13]               0
            Conv2d-9          [-1, 256, 13, 13]         884,992
             ReLU-10          [-1, 256, 13, 13]               0
           Conv2d-11          [-1, 256, 13, 13]         590,080
             ReLU-12          [-1, 256, 13, 13]               0
        MaxPool2d-13            [-1, 256, 6, 6]               0
AdaptiveAvgPool2d-14            [-1, 25

# Demonstrating sorting for AlexNet
a) Regular convolution 

In [6]:
No_layers=5 # to exlude classifier network
sw_wt = np.zeros(No_layers)
sw_bias = np.zeros(No_layers)
sw_wt_sorted = np.zeros(No_layers)
sw_bias_sorted = np.zeros(No_layers)
########## sorting features.0 ##########
_, new_indx1 = sortFullMatrix_V2(model.features[0].weight)
model.features[0].weight.data, _, sw_wt[0], sw_wt_sorted[0]  = compare_sw_sort(model.features[0].weight.data, new_indx1, -3, printsumm=True)
model.features[0].bias.data, _, sw_bias[0], sw_bias_sorted[0]  = compare_sw_sort(model.features[0].bias.data, new_indx1, -1)
# plot_dist(model.features[0][0].weight)
########## rearranging features.3 ##########
model.features[3].weight.data = model.features[3].weight[:, new_indx1, :, :]

# ########## sorting features.3 ##########
_, new_indx1 = sortFullMatrix_V2(model.features[3].weight)
model.features[3].weight.data, _, sw_wt[1], sw_wt_sorted[1]  = compare_sw_sort(model.features[3].weight.data, new_indx1, -3, printsumm=True)
model.features[3].bias.data, _, sw_bias[1], sw_bias_sorted[1]  = compare_sw_sort(model.features[3].bias.data, new_indx1, -1)
# plot_dist(model.features[0][0].weight)
########## rearranging features.3 ##########
model.features[6].weight.data = model.features[6].weight[:, new_indx1, :, :]

# ########## sorting features.6 ##########
_, new_indx1 = sortFullMatrix_V2(model.features[6].weight)
model.features[6].weight.data, _, sw_wt[2], sw_wt_sorted[2]  = compare_sw_sort(model.features[6].weight.data, new_indx1, -3, printsumm=True)
model.features[6].bias.data, _, sw_bias[2], sw_bias_sorted[2]  = compare_sw_sort(model.features[6].bias.data, new_indx1, -1)
# plot_dist(model.features[0][0].weight)
########## rearranging features.3 ##########
model.features[8].weight.data = model.features[8].weight[:, new_indx1, :, :]

# ########## sorting features.8 ##########
_, new_indx1 = sortFullMatrix_V2(model.features[8].weight)
model.features[8].weight.data, _, sw_wt[3], sw_wt_sorted[3]  = compare_sw_sort(model.features[8].weight.data, new_indx1, -3, printsumm=True)
model.features[8].bias.data, _, sw_bias[3], sw_bias_sorted[3]  = compare_sw_sort(model.features[8].bias.data, new_indx1, -1)
# plot_dist(model.features[0][0].weight)
########## rearranging features.3 ##########
model.features[10].weight.data = model.features[10].weight[:, new_indx1, :, :]

# ########## sorting features.11 ##########
_, new_indx1 = sortFullMatrix_V2(model.features[10].weight)
model.features[10].weight.data, _, sw_wt[4], sw_wt_sorted[4]  = compare_sw_sort(model.features[10].weight.data, new_indx1, -3, printsumm=True)
model.features[10].bias.data, _, sw_bias[4], sw_bias_sorted[4]  = compare_sw_sort(model.features[10].bias.data, new_indx1, -1)
# plot_dist(model.features[0][0].weight)


Switching before sorting 88294.0
Switching after sorting 79591.0
Percentage of switching reduction 9.856841914512877 %
Switching before sorting 1060179.0
Switching after sorting 965277.0
Percentage of switching reduction 8.951507245474584 %
Switching before sorting 2347256.0
Switching after sorting 2138910.0
Percentage of switching reduction 8.876151557392973 %
Switching before sorting 3048703.0
Switching after sorting 2835337.0
Percentage of switching reduction 6.998582675977293 %
Switching before sorting 2022532.0
Switching after sorting 1834344.0
Percentage of switching reduction 9.30457466185949 %


In [7]:
########## rearranging classifier.1 ##########
# Reshaping first classifier weight to rearrange prior to its sort
fc1_weight = model.classifier[1].weight
fc1_weight = fc1_weight.view(4096, 256, 6, 6)
new_fc1_weight = fc1_weight[:, new_indx1, :, :]
model.classifier[1].weight.data = new_fc1_weight.view(new_fc1_weight.shape[0], -1) 

In [8]:
# ########## sorting classifier.1 ##########
_, new_indx1 = sortFullMatrix_V2(model.classifier[1].weight)
model.classifier[1].weight.data, _, sw_wt[5], sw_wt_sorted[5]  = compare_sw_sort(model.classifier[1].weight.data, new_indx1, -1, isconv=False, printsumm=True)
model.classifier[1].bias.data, _, sw_bias[5], sw_bias_sorted[5]  = compare_sw_sort(model.classifier[1].bias.data, new_indx1, -1, isconv=False)
########## rearranging classifier.4 ##########
model.classifier[4].weight.data = model.classifier[4].weight[:, new_indx1]


Switching before sorting 32204568.0
Switching after sorting 29309605.0
Percentage of switching reduction 8.989293071715789 %


IndexError: index 5 is out of bounds for axis 0 with size 5

In [ ]:
# ########## sorting classifier.4 ##########
_, new_indx1 = sortFullMatrix_V2(model.classifier[4].weight)
model.classifier[4].weight.data, _, sw_wt[6], sw_wt_sorted[6]  = compare_sw_sort(model.classifier[4].weight.data, new_indx1, -1, isconv=False, printsumm=True)
model.classifier[4].bias.data, _, sw_bias[6], sw_bias_sorted[6]  = compare_sw_sort(model.classifier[4].bias.data, new_indx1, -1, isconv=False)
########## rearranging classifier.6 ##########
model.classifier[6].weight.data = model.classifier[6].weight[:, new_indx1]
###### No need to re order the last fc to keep the order of output as original model

In [ ]:
print('switching activity of weight before sorting: ', sw_wt)
print('switching activity of weight before sorting: ', sw_wt_sorted)
print('Percentage of switching activity of weight changes: ', (sw_wt - sw_wt_sorted)*100/sw_wt)
print('switching activity of bias after sorting: ', sw_bias)
print('switching activity of bias after sorting: ', sw_bias_sorted)
print('Percentage of switching activity of biases changes: ', (sw_bias - sw_bias_sorted)*100/sw_bias)


In [ ]:
# sample execution (requires torchvision)
from PIL import Image
from torchvision import transforms
filename = 'dog.jpg'
input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over ImageNet's 1000 classes

# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = torch.nn.functional.softmax(output[0], dim=0)


In [ ]:
# Read the categories
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
# Show top categories per image
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())